In [13]:
#OPEN DATA PENGUJIAN
import numpy as np
import pandas as pd
moon = pd.read_csv("2menit_H.csv")
datacdc =moon
datacdc =datacdc.drop(["ipsrc"], axis = 1)
datacdc =datacdc.drop(["ipdest"], axis = 1)
datacdc =datacdc.drop(["portsrc"], axis = 1)
datacdc =datacdc.drop(["portdest"], axis = 1)
datacdc =datacdc.drop(["timestamp"], axis = 1)
#print(moon.head(15))
from sklearn import preprocessing
from sklearn.preprocessing import LabelEncoder
#LABELENCODER
le = preprocessing.LabelEncoder()
# 2/3. FIT AND TRANSFORM
# use df.apply() to apply le.fit_transform to all columns
data = datacdc.apply(le.fit_transform)

#StandardSCALER
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
data_scaler=scaler.fit_transform(data)

# Normalize data (length of 1)
from sklearn.preprocessing import Normalizer
normalize = Normalizer().fit(data_scaler)

normalized=normalize.fit_transform(data_scaler)
X_test=normalized



In [14]:
def hms_string(sec_elapsed):
    h = int(sec_elapsed / (60 * 60))
    m = int((sec_elapsed % (60 * 60)) / 60)
    s = sec_elapsed % 60.
    return "{}:{:>02}:{:>05.2f}".format(h, m, s)

def konvert(prediction_test):
    prediction_string=[]
    for i  in range(len(prediction_test)):
        if(prediction_test[i]==1):
            prediction_string.append("Dos")
        elif(prediction_test[i]==2):
            prediction_string.append("Normal")
        elif(prediction_test[i]==3):
            prediction_string.append("Probe")
        elif(prediction_test[i]==4):
            prediction_string.append("R2L")
        elif(prediction_test[i]==5):
            prediction_string.append("U2R")                            
    return prediction_string

In [15]:

from sklearn.metrics import accuracy_score , cohen_kappa_score,  confusion_matrix
from time import time
#get C, Kernel, Gamma

#CODE UNTUK LEARNING
from sklearn import metrics
from sklearn.metrics import confusion_matrix , classification_report

#SAVE MODEL
from sklearn.externals import joblib

# Get model saved
joblib_file = "svm_5class.pkl"  

# Load from file
joblib_model = joblib.load(joblib_file)

start=time()
prediction_test=joblib_model.predict(X_test)
end=time()
jum=end-start

print ("Time to predict data test".format(hms_string(jum)))
print(hms_string(jum))

prediction=konvert(prediction_test)

#print(prediction)

Time to predict data test
0:00:36.02


In [16]:
df = pd.DataFrame({'class':prediction})
print(df["class"].value_counts(normalize=False))

Normal    167730
Probe      39502
Dos         9374
R2L         3233
U2R          225
Name: class, dtype: int64


In [17]:
framesall =pd.concat([moon,df], axis = 1)
framesall.tail()

,duration,protocol_type,service,flag,src_bytes,dst_bytes,land,wrong_fragment,urgent,count,...,dst_host_serror_rate,dst_host_srv_serror_rate,dst_host_rerror_rate,dst_host_srv_rerror_rate,ipsrc,portsrc,ipdest,portdest,timestamp,class
220059,0,tcp,telnet,S1,60,0,0,0,0,1,...,0.0,1.00,0.0,0.0,103.114.24.161,16854,103.114.165.38,23,2018-06-14T03:39:23,Normal
220060,0,tcp,other,S1,60,0,0,0,0,3,...,0.0,0.33,0.0,0.0,192.251.231.111,65534,202.191.3.160,8545,2018-06-14T03:39:23,Probe
220061,0,tcp,http,S1,74,0,0,0,0,19039,...,1.0,1.00,0.0,0.0,202.158.92.226,51885,128.199.171.199,80,2018-06-14T03:39:23,Dos
220062,0,udp,domain_u,SF,76,0,0,0,0,1131,...,0.5,0.00,0.0,0.0,54.165.95.142,16117,210.210.188.53,53,2018-06-14T03:39:23,Normal
220063,0,tcp,http,S1,74,0,0,0,0,401,...,1.0,1.00,0.0,0.0,120.188.66.158,50246,202.158.68.151,80,2018-06-14T03:39:23,Normal


In [18]:
a=framesall['ipsrc']
b=framesall['portsrc']
c=framesall['ipdest']
d=framesall['portdest']
e=framesall['timestamp']
f=framesall['class']
dt =pd.concat([a,b,c,d,e,f], axis = 1)
dt.tail(20)

,ipsrc,portsrc,ipdest,portdest,timestamp,class
220044,54.36.148.37,55964,202.158.92.194,80,2018-06-14T03:39:23,Normal
220045,210.210.188.123,123,175.158.50.248,11711,2018-06-14T03:39:23,Normal
220046,210.210.188.123,123,175.158.50.146,59760,2018-06-14T03:39:23,Normal
220047,64.90.22.106,48382,210.210.188.80,3389,2018-06-14T03:39:23,Probe
220048,202.158.92.226,50371,128.199.171.199,80,2018-06-14T03:39:23,Dos
220049,5.8.18.58,51118,210.210.143.102,14783,2018-06-14T03:39:23,Probe
220050,5.188.207.5,25031,210.210.188.40,143,2018-06-14T03:39:23,Dos
220051,92.63.193.25,4567,210.210.188.40,143,2018-06-14T03:39:23,Dos
220052,5.8.18.55,50978,202.158.92.196,11783,2018-06-14T03:39:23,Probe
220053,213.205.198.33,35963,202.158.92.206,25,2018-06-14T03:39:23,Normal
